In [ ]:
# import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from api_keys import g_key

In [ ]:
# reads the data created from WeatherPy
data = pd.read_csv("../outputs/city_list.csv")
# stores the latitude and longitude for the locations and humidity as a "weight" for the heatmap
locations = data["Latitude", "Longitude"]
humid = data["Humidity"]
fig = gmaps.figure()
# creating the heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, dissipating=False, max_intensity=100, point_radius=2)
fig.add_layer(heat_layer)
fig

In [ ]:
# narrowing down the data by condition until there is only 10 cities
city_data = data.loc[data["Max Temp"] > 70]
city_data = city_data.loc[city_data["Max Temp"] < 75]
city_data = city_data.loc[city_data["Humidity"] < 50]
city_data = city_data.loc[city_data["Wind Speed"] < 12]
# getting rid of unnecessary rows from the df and adding a column named "Hotel Name"
city_data = city_data[["City", "Latitude", "Longitude", "Country"]]
city_data["Hotel Name"] = ""
hotels_df = city_data
hotels_df

In [ ]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"radius" : 5000,
             "types" : "hotel",
             "key" : g_key}

# loops through the entire df
for index, row in hotels_df.iterrows():
    # stores the latitude and longitude of each city
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    # add location to the params
    params["locations"] = f"{latitude}, {longitude}"
    # make the api request
    hotel_search = requests.get(url, params=params).json()
    
    try:
        hotels_df.loc[index, "Hotel Name"] = hotel_search["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing result.")
hotels_df

In [ ]:
# setting up the hotel markers for the heatmap
info_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# stores the df row into the marker template
hotels_info = [info_template.format(**row) for index, row in hotels_df.iterrows()]
# stores the latitude and longitude for every hotel
locations = hotels_df[["Latitude", "Longitude"]]
# adding markers to the layer for each hotel
hotels_layer = gmaps.marker_layer(locations, info_box_content=hotels_info)
# adding a layer and display figure
fig.add_layer(hotel_layer)
fig